In [1]:
import sys
sys.path.append('..')
import torch
from torch import nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
%matplotlib notebook
from ipywidgets import *
import matplotlib.pyplot as plt
import plotly.express as px
from utils.functions import train_models, train_models_w_mean_var, train_and_save_results
from utils.models import BatchEnsemble
from utils.layers import BatchLinear
from utils.layers import AnchoredBatch

## Concrete Dataset

In [2]:
# Read data into a pandas dataframe
concrete = pd.read_excel('..\\data\\UCI_Regression\\2.Concrete\\Concrete_Data.xls')

concrete.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [3]:
# Set a seed for reproducibility
np.random.seed(52)

# Split into training and validation datasets
concrete_train, concrete_val = train_test_split(concrete, test_size=0.1)
concrete_train, concrete_val = concrete_train.to_numpy(), concrete_val.to_numpy()

In [4]:
# Subtract mean and divide by standard deviation
concrete_scaler = StandardScaler()
concrete_scaler.fit(X=concrete_train[:,:8], y=concrete_train[:,8])
concrete_train[:,:8] = concrete_scaler.transform(concrete_train[:,:8])

# Split the training dataset into train and val
x_train, x_test, y_train, y_test = train_test_split(concrete_train[:,:8], concrete_train[:,8],train_size=0.7)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

print(x_train_tensor.shape)
print(y_train_tensor.shape)
print(x_test_tensor.shape)
print(y_test_tensor.shape)

# Create TensorDatasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Define a batch size
batch_size = 128

# Create DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)



torch.Size([648, 8])
torch.Size([648, 1])
torch.Size([279, 8])
torch.Size([279, 1])


In [5]:
# Initalize list of Hyperparameters to try out
#hidden_layers_options = list(range(2, 6))  # 2 to 10 inclusive
hidden_layers_options = [6]
hidden_units_options = [32, 64, 128]
weight_decay_options = [0.01,0.001,0.0001,0.00001]
#weight_decay_options = [0.01,0.001,0.0001]

loss_fn = nn.GaussianNLLLoss()
optimizer = torch.optim.Adam
print(weight_decay_options)

train_and_save_results(model_name = 'batch', hidden_layers_options=hidden_layers_options, hidden_units_options=hidden_units_options, input_shape=8, loss_fn=loss_fn,optimizer=optimizer, train_loader=train_loader, test_loader=test_loader,ensemble_size=10,epochs=1000, csv_file='concrete_model_results.csv',weight_decay_options=weight_decay_options,print_frequency=100)

[0.01, 0.001, 0.0001]
Epoch: 0
-------
Loss: 1178.39404296875
Epoch: 100
-------
Loss: 6.722768147786458
Epoch: 200
-------
Loss: 4.940077702204387
Epoch: 300
-------
Loss: 3.786235531171163
Epoch: 400
-------
Loss: 3.1528915961583457
Epoch: 500
-------
Loss: 2.9289540449778237
Epoch: 600
-------
Loss: 2.8280193408330283
Epoch: 700
-------
Loss: 2.7389130194981894
Epoch: 800
-------
Loss: 2.632975419362386
Epoch: 900
-------
Loss: 2.5441601276397705

Evaluation on Test Data
------------------------
Average Test Loss: 2.592951536178589
Epoch: 0
-------
Loss: 1472.8152669270833
Epoch: 100
-------
Loss: 7.644123554229736
Epoch: 200
-------
Loss: 4.936777432759603
Epoch: 300
-------
Loss: 3.704951206843058
Epoch: 400
-------
Loss: 3.1629375219345093
Epoch: 500
-------
Loss: 2.978944460550944
Epoch: 600
-------
Loss: 2.8257725636164346
Epoch: 700
-------
Loss: 2.7327564557393393
Epoch: 800
-------
Loss: 2.6432796319325766
Epoch: 900
-------
Loss: 2.482157349586487

Evaluation on Test Data
-

In [14]:
model = BatchEnsemble(ensemble_size=10, input_shape=8, hidden_layers=3, hidden_units=128)

loss_fn = torch.nn.GaussianNLLLoss()
optimizer = torch.optim.Adam
GNLLL_result, rmse_result = train_models_w_mean_var(model=model, ensemble_type='batch', ensemble_size=10,data_noise=0,epochs=500,print_frequency=50,loss_fn=loss_fn,optimizer=optimizer,train_loader=train_loader,test_loader=test_loader,test=True,RMSE=True,weight_decay=0.0001,)

print('GaussianNLLLoss:', GNLLL_result)
print('RMSE:', rmse_result)


Epoch: 0
-------
Loss: 1052.89863170277
Epoch: 50
-------
Loss: 4.4566732103174385
Epoch: 100
-------
Loss: 2.953240914778276
Epoch: 150
-------
Loss: 2.5891949263485996
Epoch: 200
-------
Loss: 2.373821410265836
Epoch: 250
-------
Loss: 2.1957076462832363
Epoch: 300
-------
Loss: 2.094709027897228
Epoch: 350
-------
Loss: 2.0049897649071435
Epoch: 400
-------
Loss: 1.9448035088452427
Epoch: 450
-------
Loss: 1.872825557535345

Evaluation on Test Data
------------------------
Average Test Loss: 2.627743053436279
GaussianNLLLoss: 2.627743053436279
RMSE: 6.182777309417725


In [15]:
# Specify the directory where the CSV file should be saved
results_directory = '..\\results'


# Specify the CSV file name
csv_file_name = 'concrete_model_results.csv'  # Replace with your desired CSV file name

# Create the full path for the CSV file
csv_file_path = os.path.join(results_directory, csv_file_name)

# Check if the CSV file exists
try:
    # Load the existing CSV file
    df_existing = pd.read_csv(csv_file_path)
except FileNotFoundError:
    # If the file doesn't exist, create a new DataFrame
    df_existing = pd.DataFrame()

print(csv_file_path)
df_existing.to_csv(csv_file_path, mode='a', header=not df_existing.shape[0], index=False)

..\results\concrete_model_results.csv


In [23]:
import pandas as pd

# Define the configurations
hidden_layers_options = list(range(2, 11))  # 2 to 10 inclusive
hidden_units_options = [32, 64, 128]

# Initialize a list to store the results
results = []

# Loop over the configurations
for hidden_layers in hidden_layers_options:
    for hidden_units in hidden_units_options:
        # Create the model
        model = BatchEnsemble(ensemble_size=10, input_shape=8, hidden_layers=hidden_layers, hidden_units=hidden_units, output_shape=1)

        # Train the model and get the average loss on test data
        result = train_models(model=model, ensemble_type='batch', ensemble_size=10, data_noise=0, epochs=500, print_frequency=50, loss_fn=loss_fn, optimizer=optimizer, train_loader=train_loader, test_loader=test_loader, test=True, learning_rate=0.001)

        # Append the result to the list
        results.append({
            'hidden_layers': hidden_layers,
            'hidden_units': hidden_units,
            'average_loss': result
        })

# Convert the results to a DataFrame

df = pd.DataFrame(results)
list_of_dicts = df.to_dicts('records')


Epoch: 0
-------
Loss: 1.2789609161290256
Epoch: 50
-------
Loss: 0.29053364287723193
Epoch: 100
-------
Loss: 0.17948136275464838
Epoch: 150
-------
Loss: 0.16007963635704733
Epoch: 200
-------
Loss: 0.14776190708984027
Epoch: 250
-------
Loss: 0.13760600848631424
Epoch: 300
-------
Loss: 0.1303254406560551
Epoch: 350
-------
Loss: 0.12935656918720764
Epoch: 400
-------
Loss: 0.12082395702600479
Epoch: 450
-------
Loss: 0.1196495308117433

Evaluation on Test Data
------------------------
Average Test Loss: 0.11701419800519944
Epoch: 0
-------
Loss: 1.3439821059053594
Epoch: 50
-------
Loss: 0.261006236076355
Epoch: 100
-------
Loss: 0.17731957069852136
Epoch: 150
-------
Loss: 0.14439911124381152
Epoch: 200
-------
Loss: 0.145053122531284
Epoch: 250
-------
Loss: 0.10993427275256677
Epoch: 300
-------
Loss: 0.11210461909120734
Epoch: 350
-------
Loss: 0.10031452571803873
Epoch: 400
-------
Loss: 0.10265921259468253
Epoch: 450
-------
Loss: 0.08647637035359036

Evaluation on Test Data


In [27]:
list_of_dicts = df.to_dict('records')
list_of_dicts

[{'hidden_layers': 2, 'hidden_units': 32, 'average_loss': 0.11701419800519944},
 {'hidden_layers': 2, 'hidden_units': 64, 'average_loss': 0.1085217073559761},
 {'hidden_layers': 2,
  'hidden_units': 128,
  'average_loss': 0.09898357540369034},
 {'hidden_layers': 3, 'hidden_units': 32, 'average_loss': 0.12455113977193832},
 {'hidden_layers': 3, 'hidden_units': 64, 'average_loss': 0.11007597893476487},
 {'hidden_layers': 3,
  'hidden_units': 128,
  'average_loss': 0.09881208091974258},
 {'hidden_layers': 4, 'hidden_units': 32, 'average_loss': 0.11645654439926148},
 {'hidden_layers': 4, 'hidden_units': 64, 'average_loss': 0.10671028345823289},
 {'hidden_layers': 4,
  'hidden_units': 128,
  'average_loss': 0.19336654245853424},
 {'hidden_layers': 5, 'hidden_units': 32, 'average_loss': 0.12457301020622254},
 {'hidden_layers': 5, 'hidden_units': 64, 'average_loss': 0.2929555386304855},
 {'hidden_layers': 5, 'hidden_units': 128, 'average_loss': 0.9447466373443604},
 {'hidden_layers': 6, 'hidd

In [38]:
fig = px.parallel_coordinates(df, color="average_loss",
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              color_continuous_midpoint=2)
fig.show()


In [5]:
# Define the configurations
hidden_layers_options = list(range(2, 4))  # 2 to 10 inclusive
hidden_units_options = [64,128]
weight_decay_options = [0.01,0.001,0.0001,0.00001]

# Initialize a list to store the results
results = []

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam

# Loop over the configurations
for hidden_layers in hidden_layers_options:
    for hidden_units in hidden_units_options:
        for weight_decay in weight_decay_options:
            # Create the model
            model = BatchEnsemble(ensemble_size=10, input_shape=8, hidden_layers=hidden_layers, hidden_units=hidden_units, output_shape=1)

            # Train the model and get the average loss on test data
            result = train_models(model=model, ensemble_type='batch', ensemble_size=10, data_noise=0, epochs=500, print_frequency=50, loss_fn=loss_fn, optimizer=optimizer, train_loader=train_loader, test_loader=test_loader, test=True, learning_rate=0.001, weight_decay=weight_decay)

            # Append the result to the list
            results.append({
                'hidden_layers': hidden_layers,
                'hidden_units': hidden_units,
                'weight_decay': weight_decay,
                'average_loss': result
            })

# Convert the results to a DataFrame

df_concrete = pd.DataFrame(results)

Epoch: 0
-------
Loss: 1.2951505726033992
Epoch: 50
-------
Loss: 0.9436066069386222
Epoch: 100
-------
Loss: 1.01465519991788
Epoch: 150
-------
Loss: 1.0016494935209102
Epoch: 200
-------
Loss: 0.9765080538662997
Epoch: 250
-------
Loss: 0.9297991259531542
Epoch: 300
-------
Loss: 1.0272549986839294
Epoch: 350
-------
Loss: 1.0097867846488953
Epoch: 400
-------
Loss: 1.0563481775197117
Epoch: 450
-------
Loss: 1.0052577636458657

Evaluation on Test Data
------------------------
Average Test Loss: 1.0451125621795654
Epoch: 0
-------
Loss: 1.317843328822743
Epoch: 50
-------
Loss: 0.9853831258687106
Epoch: 100
-------
Loss: 0.9634399088946256
Epoch: 150
-------
Loss: 0.9796543554826216
Epoch: 200
-------
Loss: 0.9679790897802873
Epoch: 250
-------
Loss: 1.0216174504973672
Epoch: 300
-------
Loss: 1.0177323818206787
Epoch: 350
-------
Loss: 0.9425522007725455
Epoch: 400
-------
Loss: 0.9527037089521234
Epoch: 450
-------
Loss: 0.9474860321391713

Evaluation on Test Data
----------------

In [6]:
fig_2 = px.parallel_coordinates(df_concrete, color="average_loss",
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              color_continuous_midpoint=2)
fig_2.show()